# Explosions and extinctions

Major changes in a domain...

404s are not actually useful as they only show what was looked for. Try a year by year comparison?

In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm.auto import tqdm
import pandas as pd
import requests_cache
import time
import altair as alt

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('https://', HTTPAdapter(max_retries=retries))
s.mount('http://', HTTPAdapter(max_retries=retries))

alt.data_transformers.enable('json', urlpath='files')

DataTransformerRegistry.enable('json')

In [2]:

def get_total_pages(params):
    these_params = params.copy()
    these_params['showNumPages'] = 'true'
    response = s.get('http://web.archive.org/cdx/search/cdx', params=these_params, headers={'User-Agent': ''})
    return int(response.text)

def query_cdx(url, **kwargs):
    results = []
    page = 0
    params = kwargs
    params['url'] = url
    params['output'] = 'json'
    total_pages = get_total_pages(params)
    print(total_pages)
    with tqdm(total=total_pages-page) as pbar:
        while page < total_pages:
            params['page'] = page
            response = requests.get('http://web.archive.org/cdx/search/cdx', params=params, headers={'User-Agent': ''})
            print(response.url)
            response.raise_for_status()
            if page == 0:
                results = response.json()
            else:
                results += response.json()[1:]
            page += 1
            pbar.update(1)
            time.sleep(0.2)
    return results

In [3]:
results = query_cdx('*.naa.gov.au', )

299


http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=0
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=1
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=2
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=3
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=4
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=5
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=6
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=7
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=8
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=9
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=10
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=11
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=12
http://we

In [4]:
params = {
    'url': 'naa.gov.au',
    'output': 'json'
}
get_total_pages(params)

1

In [5]:
results = query_cdx('naa.gov.au/*')

86


http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=0
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=1
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=2
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=3
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=4
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=5
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=6
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=7
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=8
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=9
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=10
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&output=json&page=11
http://web.archive.org/cdx/search/cdx?url=naa.gov.au%2F%2A&out

In [6]:
df = pd.DataFrame(results[1:], columns=results[0])

In [7]:
df.head()

,urlkey,timestamp,original,mimetype,statuscode,digest,length
0,"au,gov,naa)/",19981202073240,http://www.naa.gov.au:80/,text/html,200,37G5CLZH7GK2YIFHLGD75HVBOGVXRACX,974
1,"au,gov,naa)/",19981202073240,http://www.naa.gov.au:80/,text/html,200,37G5CLZH7GK2YIFHLGD75HVBOGVXRACX,974
2,"au,gov,naa)/",19981212031222,http://www.naa.gov.au:80/,text/html,200,37G5CLZH7GK2YIFHLGD75HVBOGVXRACX,978
3,"au,gov,naa)/",19981212031222,http://www.naa.gov.au:80/,text/html,200,37G5CLZH7GK2YIFHLGD75HVBOGVXRACX,978
4,"au,gov,naa)/",19990125101445,http://www.naa.gov.au:80/,text/html,200,37G5CLZH7GK2YIFHLGD75HVBOGVXRACX,986


In [8]:
df.shape

(1285563, 7)

In [9]:
# Convert the timestamp string into a datetime object
df['date'] = pd.to_datetime(df['timestamp'], errors='coerce')
df.sort_values(by='date', inplace=True, ignore_index=True)

#df.drop_duplicates(subset=['urlkey'], keep='first', inplace=True)

In [10]:
# Convert the length from a string into an integer
df['length'] = df['length'].astype('int')

In [11]:
df.shape

(1285563, 8)

## Number of captures per year

In [12]:
df['year'] = df['date'].dt.year

# This gives an error without fillna
df['year'] = df['year'].fillna(0).astype('int')

# The error is because one of the timestamps is incorrectly formatted -- no day!
df.loc[df['year'] == 0]

,urlkey,timestamp,original,mimetype,statuscode,digest,length,date,year
1285562,"au,gov,naa)/research/factshet/fs16.html",20000800230411,http://www.naa.gov.au:80/research/FACTSHET/FS1...,text/html,200,67BZLW6T74JVOB56RL7NCVY4KYURQWLG,1410,NaT,0


We could just feed to whole dataframe to Altair for visualisation, but that's not very efficient. Let's transform it with Pandas first.

In [13]:
# Group by year and status code
df_year_code = df.loc[df['year'] > 0].groupby('year')['statuscode'].value_counts().to_frame().rename(columns={'statuscode': 'total'}).reset_index()
df_year_code.head()

,year,statuscode,total
0,1998,200,18
1,1999,200,3639
2,1999,404,72
3,1999,302,2
4,2000,200,9629


In [14]:
alt.Chart(df_year_code).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color='statuscode:N'
).properties(width=700)

alt.Chart(...)

In [15]:
# Group by year and mimetype
df_year_type = df.loc[df['year'] > 0].groupby('year')['mimetype'].value_counts().to_frame().rename(columns={'mimetype': 'total'}).reset_index()
df_year_type.head()

,year,mimetype,total
0,1998,text/html,18
1,1999,text/html,3673
2,1999,im,32
3,1999,unk,8
4,2000,text/html,9776


In [16]:
# It would be good to limit to top mimetypes, grouping all the low ranked values together in Altair
# Something like this? https://altair-viz.github.io/gallery/top_k_with_others.html
# Can't get it to work...

alt.Chart(df_year_type).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color='mimetype:N'
).properties(width=700)

alt.Chart(...)

In [17]:
# Only show top-ten mimetypes

alt.Chart(df_year_type.loc[df_year_type['mimetype'].isin(df['mimetype'].value_counts()[:10].index.to_list())]).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color=alt.Color('mimetype:N', scale=alt.Scale(scheme='category20b'))
).properties(width=700)

alt.Chart(...)

In [18]:
# Normalise mimetypes by grouping everything outside the top ten into 'Other'
top_types = df['mimetype'].value_counts()[:9].index.to_list()
df_year_type['top_types'] = df_year_type['mimetype'].apply(lambda x: x if x in top_types else 'Other')

In [19]:
alt.Chart(df_year_type).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color=alt.Color('top_types:N', scale=alt.Scale(scheme='category20b'))
).properties(width=700)

alt.Chart(...)

## Mimetypes and extensions

In [20]:
df['mimetype'].value_counts().index.to_list()

['text/html',
 'image/jpeg',
 'image/gif',
 'warc/revisit',
 'application/pdf',
 'text/plain',
 'application/octet-stream',
 'image/png',
 'text/css',
 'application/msword',
 'application/x-javascript',
 'audio/mpeg',
 'unk',
 'application/rtf',
 'image/svg+xml',
 'im',
 'text/xml',
 'application/vnd.ms-powerpoint',
 'image/x-icon',
 'application/vnd.ms-pps',
 'image/bmp',
 'application/x-zip-compressed',
 'application/vnd.ms-excel',
 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'font/woff2',
 'application/x-shockwave-flash',
 'application/javascript',
 'UNKNOWN',
 'application/x-msmetafile',
 'font/woff',
 'application/vnd.ms-fontobject',
 'video/quicktime',
 'video/x-ms-wmv',
 'application/zip',
 'image/tiff',
 'application/vnd',
 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
 'application/vnd.ms-excel.sheet.macroenabled.12',
 'application/vnd.openxmlformats-officedocument.presentationml.slideshow']

In [21]:
df['extension'] = df['original'].str.extract(r'\.(\w+)$')

In [22]:
df.loc[df['mimetype'] == 'application/pdf']['extension'].value_counts()[:25]

pdf    48285
PDF      425
Name: extension, dtype: int64

In [23]:
df.loc[df['mimetype'] == 'text/html']['extension'].value_counts()[:25]

htm      324118
html     252882
aspx     132237
txt       10879
gif        4581
2          2367
jpg        1302
pdf        1028
asp         650
1           386
ico         278
3           210
css         208
woff        206
woff2       204
HTM         198
0           160
png         137
doc         104
mso          81
xml          75
au           67
idq          61
js           60
svg          57
Name: extension, dtype: int64

In [24]:
df['mimetype'].value_counts()[:25]

text/html                                                                  796672
image/jpeg                                                                 144261
image/gif                                                                  106419
warc/revisit                                                                87183
application/pdf                                                             48710
text/plain                                                                  40148
application/octet-stream                                                    26694
image/png                                                                    8305
text/css                                                                     5895
application/msword                                                           5395
application/x-javascript                                                     3170
audio/mpeg                                                                   2623
unk             

## Change by year

How many pages were added or removed each year.

In [25]:
# Get unique urls for each year
df_years = df.loc[df['statuscode'].isin(['200', '-', '301', '302'])].drop_duplicates(subset=['year', 'urlkey'])

In [26]:
year_totals = df_years['year'].value_counts().to_frame().reset_index()
year_totals.columns = ['year', 'total']
year_totals.head()

,year,total
0,2019,19149
1,2011,18840
2,2007,18422
3,2012,14532
4,2006,14204


In [27]:
alt.Chart(year_totals).mark_bar().encode(
    x='year:N',
    y='total'
).properties(width=700)

alt.Chart(...)

In [28]:
df_years['top_types'] = df_years['mimetype'].apply(lambda x: x if x in top_types else 'Other')
year_type_totals = df_years.loc[df_years['year'] > 0].groupby('year')['top_types'].value_counts().to_frame().rename(columns={'top_types': 'total'}).reset_index()
alt.Chart(year_type_totals).mark_bar().encode(
    x='year:N',
    y='total:Q',
    color=alt.Color('top_types:N', scale=alt.Scale(scheme='category20b'))
).properties(width=700)

alt.Chart(...)

In [29]:
df_years.loc[df['mimetype'].isin(['text/html', 'warc/revisit'])].groupby('urlkey')['year'].count().mean()

2.833253540793936

In [30]:
import numpy as np
df_year_range = df_years.groupby(['urlkey', 'top_types'])['year'].agg([np.max, np.min])
df_year_range.head()

amax  amin
urlkey                         top_types                           
au,gov,naa)/                   application/octet-stream  2020  2020
                               text/html                 2019  1998
au,gov,naa)/"%20target="_blank text/html                 2015  2015
au,gov,naa)/&&a.type!==        text/html                 2012  2012
au,gov,naa)/*/*                text/html                 2012  2012

In [31]:
df_year_range['range'] = df_year_range['amax'] - df_year_range['amin']
df_year_range.reset_index(inplace=True)

In [32]:
df_year_range['range'].mean()

2.4606855918585966

In [33]:
df_year_range.groupby('top_types')['range'].mean()

top_types
Other                       1.338938
application/octet-stream    0.680412
application/pdf             2.554468
image/gif                   3.145576
image/jpeg                  3.178743
image/png                   1.948882
text/css                    2.042424
text/html                   2.382658
text/plain                  1.975610
warc/revisit                0.984007
Name: range, dtype: float64

In [34]:
alt.Chart(df_year_range).mark_bar().encode(
    y='top_types:N',
    x='mean(range):Q'
)

alt.Chart(...)

In [35]:
years = []
for year in range(1998, 2021):
    if year == 1998:
        added = df_years.loc[df_years['year'] == year].shape[0]
        removed = 0
    else:
        this_year = df_years.loc[df_years['year'] == year]['urlkey']
        last_year = df_years.loc[df_years['year'] == year - 1]['urlkey']
        added = this_year.loc[~this_year.isin(last_year)].shape[0]
        removed = last_year.loc[~last_year.isin(this_year)].shape[0]
    years.extend([{'year': year, 'change': added, 'direction': 'added'}, {'year': year, 'change': 0 - removed, 'direction': 'removed'}])
    
    

In [36]:
alt.Chart(pd.DataFrame(years)).mark_bar().encode(
    x='year:N',
    y='change:Q',
    color='direction:N'
)

alt.Chart(...)

In [37]:
df_years

,urlkey,timestamp,original,mimetype,statuscode,digest,length,date,year,extension,top_types
0,"au,gov,naa)/",19981202073240,http://www.naa.gov.au:80/,text/html,200,37G5CLZH7GK2YIFHLGD75HVBOGVXRACX,974,1998-12-02 07:32:40,1998,NaN,text/html
2,"au,gov,naa)/index.htm",19981202183753,http://www.naa.gov.au:80/index.htm,text/html,200,WU2HBZGPXOSEGZ7LE7X75JEIXWRK4CSU,909,1998-12-02 18:37:53,1998,htm,text/html
4,"au,gov,naa)/govserv/techpub/elecrecd/ker6.html",19981202202255,http://www.naa.gov.au:80/govserv/techpub/elecr...,text/html,200,MZZ2BFSRJKQRAOL5JU3PZXI7MAOTKICY,5945,1998-12-02 20:22:55,1998,html,text/html
6,"au,gov,naa)/govserv/techpub/manelrec/contents.htm",19981205201538,http://www.naa.gov.au:80/govserv/techpub/manel...,text/html,200,NR35LJUC5B2PIEUPZI3Z5F5BEHJBHQER,1310,1998-12-05 20:15:38,1998,htm,text/html
8,"au,gov,naa)/govserv/techpub/elecrecd/keepinger...",19981205211942,http://www.naa.gov.au:80/govserv/techpub/elecr...,text/html,200,426Z47ESTDXUEWXU5ONI3C35JIBDW3GH,2133,1998-12-05 21:19:42,1998,html,text/html
...,...,...,...,...,...,...,...,...,...,...,...
1285521,"au,gov,naa)/naaresources/publications/research...",20200326195454,http://www.naa.gov.au/naaresources/publication...,unk,301,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,405,2020-03-26 19:54:54,2020,pdf,Other
1285528,"au,gov,naa)/naaresources/css/ie9.css",20200328000849,http://www.naa.gov.au/naaresources/css/ie9.css,warc/revisit,-,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,464,2020-03-28 00:08:49,2020,css,warc/revisit
1285529,"au,gov,naa)/naaresources/images/naalogoblue.jpg",20200328001224,http://www.naa.gov.au/naaresources/images/NAAL...,unk,301,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,385,2020-03-28 00:12:24,2020,jpg,Other
1285530,"au,gov,naa)/naaresources/js/hoverintent.js",20200328001453,http://www.naa.gov.au/naaresources/js/hoverInt...,unk,301,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,382,2020-03-28 00:14:53,2020,js,Other


In [38]:
df.loc[20912].to_list()

['au,gov,naa)/publications/fact_sheets/fs123.html',
 '20010426003132',
 'http://www.naa.gov.au:80/publications/fact_sheets/FS123.html',
 'text/html',
 '200',
 'LKLNRCH75XP6CUIA5FGWYQFUPNONC774',
 6074,
 Timestamp('2001-04-26 00:31:32'),
 2001,
 'html']

In [39]:
df.loc[df['urlkey'] == 'au,gov,naa)/publications/fact_sheets/fs123.html'][200:]

,urlkey,timestamp,original,mimetype,statuscode,digest,length,date,year,extension
639848,"au,gov,naa)/publications/fact_sheets/fs123.html",20070610092722,http://www.naa.gov.au/publications/fact_sheets...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5708,2007-06-10 09:27:22,2007,html
653103,"au,gov,naa)/publications/fact_sheets/fs123.html",20070611110745,http://www.naa.gov.au//Publications/fact_sheet...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5712,2007-06-11 11:07:45,2007,html
653104,"au,gov,naa)/publications/fact_sheets/fs123.html",20070611110745,http://www.naa.gov.au//Publications/fact_sheet...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5712,2007-06-11 11:07:45,2007,html
666019,"au,gov,naa)/publications/fact_sheets/fs123.html",20070612112119,http://www.naa.gov.au///Publications/fact_shee...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5714,2007-06-12 11:21:19,2007,html
666020,"au,gov,naa)/publications/fact_sheets/fs123.html",20070612112119,http://www.naa.gov.au///Publications/fact_shee...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5714,2007-06-12 11:21:19,2007,html
752150,"au,gov,naa)/publications/fact_sheets/fs123.html",20070807115743,http://www.naa.gov.au:80/Publications/fact_she...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5717,2007-08-07 11:57:43,2007,html
752151,"au,gov,naa)/publications/fact_sheets/fs123.html",20070807115743,http://www.naa.gov.au:80/Publications/fact_she...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5731,2007-08-07 11:57:43,2007,html
752797,"au,gov,naa)/publications/fact_sheets/fs123.html",20070808084857,http://www.naa.gov.au:80/publications/fact_she...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5727,2007-08-08 08:48:57,2007,html
752798,"au,gov,naa)/publications/fact_sheets/fs123.html",20070808084857,http://www.naa.gov.au:80/publications/fact_she...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5713,2007-08-08 08:48:57,2007,html
753627,"au,gov,naa)/publications/fact_sheets/fs123.html",20070811022549,http://www.naa.gov.au:80/publications/fact_she...,text/html,200,QUD5GAR6ATK5X3YXW3IFUOIAHKRD3352,5716,2007-08-11 02:25:49,2007,html


In [40]:
response = requests.get('https://web.archive.org/web/20200316010205id_/https://www.naa.gov.au/information-management/managing-information-and-records/types-information/social-media/index.aspx', headers={'User-Agent': ''})
response.status_code

404

In [ ]:
df.loc[df['mimetype'] == 'warc/revisit'].head(20)

In [ ]:
urls.loc[urls['year'] == 1998]['urlkey'].to_frame()

In [ ]:
df.loc[df['statuscode'] == '302']

In [ ]:
df.loc[(df['date'] >= '2007-01-01') & (df['date'] <= '2007-12-31')]

In [ ]:
alt.Chart(df.loc[(df['date'] >= '2007-01-01') & (df['date'] <= '2007-12-31')]).mark_bar().encode(
    x = 'month(date):T',
    y = 'count():Q'
)

In [ ]:
from urllib.parse import urlparse
import re

paths = []
for row in df.loc[(df['date'] >= '2007-01-01') & (df['date'] <= '2007-12-31')].itertuples():
    parsed = urlparse(row.original)
    paths += [w for w in re.split(r'\W+', parsed[2]) if w != '']

In [ ]:
from collections import Counter
c = Counter(paths)
c.most_common(50)

## Mimetypes and file extensions

In [ ]:
df['extension'] = df['original'].str.extract(r'\.(\w+)$')

In [ ]:
df['extension'].value_counts()[:25]

In [ ]:
df_formats = df.loc[df['extension'].isin(['html', 'css', 'jpg', 'gif', 'htm', 'pdf', 'png', 'mp3', 'txt', 'ppt', 'swf', 'doc'])]

In [ ]:
alt.Chart(df_formats).mark_bar().encode(
    x = 'year(date):T',
    y = 'count():Q',
    color = 'extension'
).properties(width=700)

In [ ]:
df.loc[(df['date'] >= '2012-01-01') & (df['date'] <= '2012-12-31') & (df['extension'] == 'pdf')]

In [ ]:
df.loc[df['original'].str.contains('/dsp/rft/model.pdf')]

In [ ]:
df.loc[(df['date'] >= '2000-01-01') & (df['date'] <= '2000-12-31') & (df['extension'] == 'gif')]

In [ ]:
df.loc[df['extension'] == 'pdf']['urlkey'].to_list()